# Descarga variables ADS-SGR Terridata

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from time import sleep
import zipfile
from zipfile import ZipFile
import os
import glob
from glob import glob
from os import remove

In [3]:
Terridata = pd.read_excel('/Users/juancamiloperdomo/Documents/CNC/ADS-SGR/Base/Excel/Listado_variables.xlsx')
Indicadores = Terridata['variable'].astype(str)
df = pd.read_stata('/Users/juancamiloperdomo/Documents/CNC/ADS-SGR/Base/Stata/CEDE_GESPROY_Hcoca0.dta')
base = df[['codmpio','AÑO']]

In [4]:
for i in range(1):
    try:
        # Se define la variable que se va a descargar
        Variable = Indicadores[i]
        # Se entra a Terridata y se descarga la variable
        url = 'https://terridata.dnp.gov.co/index-app.html#/descargas'
        options = webdriver.ChromeOptions()
        options.add_argument('--incognito')
        driver = webdriver.Chrome(executable_path= '/Users/juancamiloperdomo/Documents/CNC/ADS-SGR/Python/chromedriver', options=options)
        driver.get(url)
        #descargas = driver.find_element_by_xpath('//a[@href="index-app.html#/descargas"]')
        #descargas.click()
        descarga_por_indicador = driver.find_element_by_xpath('//div[@ng-click="downloadXVariable()"]')
        descarga_por_indicador.click()
        sleep(5)
        indicador = driver.find_element_by_xpath('//div[@class="ui-grid-header-cell ui-grid-clearfix ng-scope ng-isolate-scope ui-grid-coluiGrid-004K"]/div[@role="columnheader"]/div[@ui-grid-filter=""]/div[@class="ui-grid-filter-container ng-scope"]/div[@class="ng-scope"]/input[@type="text"]')
        indicador.send_keys(Variable)
        sleep(5)
        archivo = driver.find_element_by_xpath('//div[@ng-repeat="(colRenderIndex, col) in colContainer.renderedColumns track by col.uid"]/div[@class="ng-scope"]')
        archivo.click()
        print('1: Descaraga la variable ' + Variable)
        sleep(5)
        driver.quit()
    except Exception as e:
        print(e)
        driver.close()
    # Se entra a la carpeta comprimida para sacar el archivo que tiene la información de la variable
    try:
        filepath = '/Users/juancamiloperdomo/Downloads/'
        Nombre = str(glob(filepath+"*.zip"))
        if len(Nombre) == 72:
            nom_var = Nombre[37:70]
        elif len(Nombre) == 73:
            nom_var = Nombre[37:71]
        else:
            nom_var = Nombre[37:72]
        archive = zipfile.ZipFile(filepath+nom_var, 'r')
        if len(Nombre) == 72:
            xlfile = archive.open(nom_var[0:29])
        elif len(Nombre) == 73:
            xlfile = archive.open(nom_var[0:30])
        else:
            xlfile = archive.open(nom_var[0:31])
        print('2: Se tomó el archivo de la variable '+ Variable)
    except Exception as e:
        print(e)   
    # Se crea una base con la información de la variable
    try:
        df = pd.read_excel(xlfile, skiprows=[1])
        print('3: Se abrió la base de la variable ' + Variable)
        info_imp = df[[str('Código Entidad'), 'Año', 'Dato Numérico']]
        info_imp.columns = ['codmpio', 'AÑO', Variable]
        #Eliminar el archivo descargado
        if len(Nombre) == 72:
            remove(Nombre[2:70])
        elif len(Nombre) == 73:
            remove(Nombre[2:71])
        else:
            remove(Nombre[2:72])
        print('4: Se eliminó el archivo de la variable')
    except Exception as e:
        print(e)
    # Se unen los DataFrame
    base_completa = base.merge(info_imp, on=['codmpio', 'AÑO'], how="left")

1: Descaraga la variable % de ingresos corrientes destinados a funcionamiento
2: Se tomó el archivo de la variable % de ingresos corrientes destinados a funcionamiento
3: Se abrió la base de la variable % de ingresos corrientes destinados a funcionamiento
4: Se eliminó el archivo de la variable


In [ ]:
base_completa.to_excel('base.xlsx')

In [5]:
base_completa

,codmpio,AÑO,% de ingresos corrientes destinados a funcionamiento
0,5001,2002,"53,78"
1,5001,2003,"49,32"
2,5001,2004,"48,29"
3,5001,2005,"42,74"
4,5001,2006,"39,57"
...,...,...,...
21105,99773,2015,"53,36"
21106,99773,2016,"86,56"
21107,99773,2017,"88,70"
21108,99773,2018,NaN
